In [1]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import random


In [6]:
#  read txt file and convert to 1000 by 2 string 2D list

file = open('badges/train/train.names.txt','r')
lines = file.readlines()
rows, cols = (len(lines), 2) 
txtarr = [[0]*cols]*rows 
for i,row in enumerate(lines):
    txtarr[i] = row.split()

# convert to feature of 1000 by 260 X_train

X_train = convert_X_txt(lines,txtarr) 
y_train = np.load('badges/train/y_train_badges.npy')
  
    
# load and convert test files
    
file = open('badges/test/test_names_badges.txt','r')
lines = file.readlines()
rows, cols = (len(lines), 2) 
txtarr = [[0]*cols]*rows 
for i,row in enumerate(lines):
    txtarr[i] = row.split()

X_test = convert_X_txt(lines,txtarr)
y_test = np.load('badges/test/y_test_badges.npy')


In [7]:
file = open('badges/train/train.names.txt','r')
lines = file.readlines()
rows, cols = (len(lines), 2) 
txtarr = [[0]*cols]*rows 
for i,row in enumerate(lines):
    txtarr[i] = row.split()
    print(txtarr[i])


['ora', 'feirberg']
['vonwun', 'itzhakova']
['yehudit', 'naim']
['lars', 'bouskilla']
['masha', 'schaffer']
['lisa', 'stern']
['sonya', 'raz']
['hadas', 'gamliel']
['zeruya', 'kwek']
['chunnan', 'ikeda']
['galia', 'diamant']
['andrey', 'bhanu']
['batya', 'pillaipakkamnatt']
['ronitt', 'katzav']
['nicolas', 'stephan']
['wee', 'gur']
['randolph', 'illouz']
['maya', 'mesika']
['ronitt', 'dunietz']
['arun', 'seung']
['julia', 'hong']
['hanna', 'adar']
['lia', 'kadman']
['darko', 'sokolova']
['daniel', 'verbeurgt']
['sean', 'daley']
['anselm', 'tsur']
['haika', 'klein']
['mary', 'ashram']
['shifra', 'guertin']
['yoav', 'netzer']
['bir', 'horn']
['ksenia', 'lotan']
['idit', 'kramer']
['shaul', 'murphy']
['jason', 'grumberg']
['kan', 'tornai']
['zhixiang', 'abe']
['claude', 'boianjiu']
['shifra', 'herlitz']
['ayelet', 'opitz']
['chris', 'applebaum']
['masha', 'yiv']
['joe', 'rosca']
['toshiyasu', 'mayoraz']
['hasya', 'decatur']
['merrick', 'redman']
['ekaterina', 'dobrin']
['barno', 'indjic']

In [9]:
np.save('checkme.npy',X_train)
np.savetxt('checkme.csv', X_train, delimiter=',')

In [10]:
hybrid_training_accuracy, hybrid_testing_accuracy, hybrid_model = train_hybrid_model(X_train,y_train,X_test,y_test)

In [11]:
print(hybrid_training_accuracy)
print(hybrid_testing_accuracy)


0.631
0.545


In [3]:
def alphabet_loc(char):
#     returns the position of the character in the alphabet in a 1X26 of 0,1 array, eg 1 at array[3] means char is a C
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    index = alphabet.index(char)
    result = np.zeros([26,])
    result[index] = 1
    return result

In [4]:
def convert_X_txt(lines,txtarr):
    X = np.zeros([len(lines),260])
    for i,row in enumerate(txtarr):
        string1 = txtarr[i][0]
        string2 = txtarr[i][1]
        feature_row = np.zeros(260,)
        for j in range(5):
            if j < len(string1):
                feature_row[j*26:(j+1)*26] = alphabet_loc(string1[j])
        for j in range(5):
            if j < len(string2):
                feature_row[(j+5)*26:(j+6)*26] = alphabet_loc(string2[j])
        X[i,:] = feature_row
    return X

In [5]:
def train_hybrid_model(X_train, y_train, X_test, y_test):
    # hybrid model full train and test set

    M = X_train.shape[1]      #number of features
    tree_num = 100      #number of trees as features
    # initialize a N by tree_num as new X_train made of outputs from 100 trees
    X_train_treefeature = np.empty([X_train.shape[0],tree_num])   
    forest = []
    feature_indexes = np.empty([tree_num,int(M/2)])
    # train 100 stumps per example as features
    for i in range(tree_num):
        index_vec = list(range(M))
        for j in range(int(M/2)):
            index_vec.pop(random.randrange(0,int(M/2)))
        X_train_sub = np.delete(X_train,index_vec,1)
        newstump = DecisionTreeClassifier(criterion='entropy',max_depth=4)
        newstump = newstump.fit(X_train_sub,y_train)
        X_train_treefeature[:,i] = newstump.predict(X_train_sub)   #output x predictions
        forest.append(newstump)     #output new tree stump classifier
        feature_indexes[i,:] = index_vec      #output feature indexes for this tree

    # apply the stump modified X_train to the sgd classifier
    sgd_hybrid = SGDClassifier(loss = 'log', max_iter = 10000)
    sgd_hybrid = sgd_hybrid.fit(X_train_treefeature,y_train)

    # convert X_test to treefeature and test accuracy on test set
    X_test_treefeature = np.empty([X_test.shape[0],tree_num])
    index_vec = feature_indexes.astype(int)

    for i in range(tree_num):
        X_test_sub = np.delete(X_test,index_vec[i,:],1)
        X_test_treefeature[:,i] = forest[i].predict(X_test_sub)
    sgd_hybrid.predict(X_test_treefeature)
    
    train_accuracy = sgd_hybrid.score(X_train_treefeature,y_train)
    test_accuracy = sgd_hybrid.score(X_test_treefeature,y_test)
    
    return train_accuracy, test_accuracy, sgd_hybrid